In [1]:
import os
import random
import numpy as np
import keras
import tensorflow as tf
import h5py
from l3embedding.model import *
from l3embedding.audio import pcm2float
import umap
from sklearn.manifold import TSNE

Using TensorFlow backend.


In [2]:
graph = tf.get_default_graph()
weight_path = 'models/cnn_l3_melspec2_recent/model_best_valid_accuracy.h5'
audio_model = load_embedding(weight_path, model_type = 'cnn_L3_melspec2', embedding_type = 'audio', \
                             pooling_type = 'short', kd_model=False, tgt_num_gpus = 1)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


/home/sk7898/miniconda3/envs/l3embedding-new-cpu/lib/python3.6/site-packages/librosa/filters.py:261: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


fmax: None
<bound method Container.summary of <keras.engine.training.Model object at 0x2b695f61ed30>>


In [4]:
def get_embedding(data, method, emb_length=None, neighbors=10, min_dist=0.3, iterations=300):
    if len(data) == 0:
        raise ValueError('Data is empty!')
    if emb_length is None:
        raise ValueError('Reduced embedding dimension was not provided!')

    if method == 'umap':
        embedding = umap.umap_.UMAP(n_neighbors=neighbors, min_dist=min_dist, \
                                    n_components=emb_length).fit_transform(data)
    elif method == 'tsne':
        embedding = TSNE(perplexity=neighbors, n_components=emb_length, n_iter=iterations).fit_transform(data)
    else:
        raise ValueError('Reduction method technique should be either `umap` or `tsne`!')
    
    return embedding

In [5]:
def data_generator(data_dir, reduced_emb_len, output_dir, reduction_method='umap', neighbors=10, min_dist=0.3, tsne_iter=300, \
                   batch_size=1024, random_state=20180123, start_batch_idx=None):

    if data_dir == output_dir:
        raise ValueError('Output path should not be same as Data path to avoid overwriting data files!')
    random.seed(random_state)

    batch = None
    global graph
    global audio_model
    curr_batch_size = 0
    batch_idx = 0
    keys = ['audio']

    for fname in os.listdir(data_dir):
        print('Filename: ', fname)
        batch_path = os.path.join(data_dir, fname)
        blob_start_idx = 0
        blob_end_idx = 0

        blob = h5py.File(batch_path, 'r')
        blob_size = len(blob['label'])

        old_embeddings = np.zeros((blob_size, 512), dtype=np.float32)
        embeddings = np.zeros((blob_size, reduced_emb_len), dtype=np.float32)
        embedding_out_path = os.path.join(output_dir, fname)

        if reduction_method == 'umap':
            embedding_key = reduction_method + '_batch_' + str(batch_size) + '_k_' + str(neighbors) + '_dist_' + str(min_dist)
        elif reduction_method == 'tsne':
            embedding_key = reduction_method + '_batch_' + str(batch_size) + '_k_' + str(neighbors) + '_iter_' + str(tsne_iter)

        while blob_start_idx < blob_size:
            print('in while')
            blob_end_idx = min(blob_start_idx + batch_size - curr_batch_size, blob_size)

            # If we are starting from a particular batch, skip computing all of
            # the prior batches
            if start_batch_idx is None or batch_idx >= start_batch_idx:
                if batch is None:
                    batch = {k:blob[k][blob_start_idx:blob_end_idx]
                             for k in keys}
                else:
                    for k in keys:
                        batch[k] = np.concatenate([batch[k],
                                                   blob[k][blob_start_idx:blob_end_idx]])

            curr_batch_size += blob_end_idx - blob_start_idx

            if blob_end_idx == blob_size:
                blob.close()

            if curr_batch_size == batch_size:
                # If we are starting from a particular batch, skip yielding all
                # of the prior batches
                if start_batch_idx is None or batch_idx >= start_batch_idx:
                    # Convert audio to float
                    batch['audio'] = pcm2float(batch['audio'], dtype='float32')
                    # Get the embedding layer output from the audio_model and flatten it to be treated as labels for the student audio model
                    with graph.as_default():
                        teacher_embedding = audio_model.predict(batch['audio'])

                    old_embeddings[blob_start_idx:blob_end_idx, :] = teacher_embedding
                    embeddings[blob_start_idx:blob_end_idx,:] = get_embedding(teacher_embedding, reduction_method, \
                                                                              emb_length=reduced_emb_len, neighbors=neighbors, \
                                                                              min_dist=min_dist, iterations=tsne_iter)

                blob_start_idx = blob_end_idx
                batch_idx += 1
                curr_batch_size = 0
                batch = None
                print('One batch done')

        with h5py.File(embedding_out_path, 'w') as f:
            f.create_dataset(embedding_key, data=embeddings)
            f.create_dataset('embedding', data=old_embeddings)
            f.close()

In [6]:
data_dir = '/scratch/sk7898/temp_data'
reduced_emb_len = 256
output_dir = '/scratch/sk7898/reduced_embeddings'

data_generator(data_dir, reduced_emb_len, output_dir, reduction_method='umap', neighbors=10, min_dist=0.3, tsne_iter=300, \
               batch_size=1024, random_state=20180123)

Filename:  20180261_7_35.h5
in while


/home/sk7898/miniconda3/envs/l3embedding-new-cpu/lib/python3.6/site-packages/umap/spectral.py:229: UserWarning: Embedding a total of 4 separate connected components using meta-embedding (experimental)
  n_components


One batch done
Filename:  20180261_7_34.h5
in while
One batch done


In [7]:
for fname in os.listdir(output_dir):
    batch_path = os.path.join(output_dir, fname)

    blob = h5py.File(batch_path, 'r')
    
    for key in blob.keys():
          print('Key {} has output shape {}'.format(key, blob[key].shape)) 

KeysView(<HDF5 file "20180261_7_35.h5" (mode r)>)
Key embedding has output shape (1024, 512)
Key umap_batch_1024_k_10_dist_0.3 has output shape (1024, 256)
KeysView(<HDF5 file "20180261_7_34.h5" (mode r)>)
Key embedding has output shape (1024, 512)
Key umap_batch_1024_k_10_dist_0.3 has output shape (1024, 256)
